Steps:


**1. Create data model**  
    - **attributes**  
    - **flags**  
    - **groups** 

2. Process data model
3. Explore networks
4. Generate AI reports

# 1. Create data model

In [ ]:
#%pip install ucimlrepo
import sys
from ucimlrepo import fetch_ucirepo
import polars as pl


sys.path.append("../..")
computer_hardware  = fetch_ucirepo(id=29)


input_dataframe = pl.DataFrame(computer_hardware.data.original)
pl.Config.set_fmt_str_lengths(200)
input_dataframe.head()

entity_id_column = "VendorName"
columns_to_link = ["ModelName", "MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"]

## Add Entity-Attributes

In [ ]:
from toolkit.risk_networks.prepare_model import build_model_with_attributes
from toolkit.risk_networks.config import ENTITY_LABEL

graph = build_model_with_attributes(input_dataframe, entity_id_column, columns_to_link)
all_nodes = graph.nodes()
entity_nodes = [
    node for node in all_nodes if node.startswith(ENTITY_LABEL)
]

num_entities = len(entity_nodes)
num_attributes = len(all_nodes) - num_entities
num_edges = len(graph.edges())

print(
    f"*Number of entities*: {num_entities}\n*Number of attributes*: {num_attributes}\n*Number of links*: {num_edges}"
)


## Add flags (Optional)

In [ ]:
import numpy as np
unique_entities = input_dataframe[entity_id_column].unique().to_list()

flags_df = pl.DataFrame({
    "VendorName": pl.Series(unique_entities),
    "Flags": pl.Series(np.random.randint(0, 2, len(unique_entities))),
})


In [ ]:
from toolkit.risk_networks.config import FlagAggregatorType
from toolkit.risk_networks.prepare_model import get_flags


flag_agg = FlagAggregatorType.Count.value
value_cols = ["Flags"]

(integrated_flags, max_entity_flags, mean_flagged_flags) = get_flags(flags_df, entity_id_column, flag_agg, value_cols)

num_flags = len(integrated_flags)
print(f"*Number of flags*: {num_flags}")



### Summary

In [ ]:
num_groups=0
print(f"*Number of entities*: {num_entities}\n*Number of attributes*: {num_attributes}\n*Number of links*: {num_edges}\n*Number of flags*: {num_flags}\n*Number of groups*: {num_groups}")
